In [1]:
import random
import networkx as nx
from matplotlib import pyplot as plt
from collections import Counter
import scipy as sc
from scipy.linalg import expm

import math
from utils.plotTools import plot_qwak
import os
import ast
import numpy as np
import json
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression

In [ ]:
import cupy as cp
import networkx as nx

def probability_distribution(M_t, init_state):
    p0 = cp.zeros(len(M_t))
    p0[init_state] = 1
    p_t = cp.round(cp.dot(M_t, p0), 5)
    return p_t

def efficient_transition_matrix(eigenvalues, eigenvectors, t):
    D_t = cp.diag(cp.exp(eigenvalues * t))
    return cp.matmul(cp.matmul(eigenvectors, D_t), cp.linalg.inv(eigenvectors))

def first_passage_time2(G, gamma, start, end, delta_t):
    D = cp.diag(cp.array(list(dict(G.degree()).values())))
    A = cp.array(nx.to_numpy_array(G))
    L = -gamma * (D - A)

    # Compute eigenvalues and eigenvectors using CuPy
    eigenvalues, eigenvectors = np.linalg.eigh(L)
    # print(type(eigenvalues))

    current_state = start
    time_elapsed = delta_t
    
    while current_state != end:
        M_t = efficient_transition_matrix(eigenvalues, eigenvectors, time_elapsed)
        p_t = probability_distribution(M_t, current_state)
        
        neighbors = list(G.neighbors(current_state)) + [current_state]
        transition_probabilities = p_t[neighbors].get()  # Convert to NumPy array for random choice

        if transition_probabilities.sum() > 0:
            transition_probabilities /= transition_probabilities.sum()
        else:
            transition_probabilities = cp.ones(len(neighbors)) / len(neighbors)

        next_state = np.random.choice(neighbors, p=transition_probabilities)
        current_state = next_state.item()  # Convert CuPy scalar to Python scalar
        time_elapsed += delta_t

    return time_elapsed


In [ ]:
import time

def average_hitting_time_for_range2(n_range, samples, gamma, delta_t):
    """
    Calculates the average hitting time for graphs of sizes within n_range,
    performing a specified number of samples for each graph size.

    :param n_range: Range of graph sizes to test (e.g., range(5, 10))
    :param samples: Number of samples to perform for each graph size
    :param gamma: Transition rate parameter for the random walk
    :param delta_t: Time step for the random walk
    :return: List with average hitting times for each graph size
    """
    avg_hitting_times = []

    for n in n_range:
        # print(n)
        total_time = 0
        end_vertex = n // 2

        for _ in range(samples):
            G = nx.cycle_graph(n)
            start_vertex = 0

            time_taken = first_passage_time2(G, gamma, start_vertex, end_vertex, delta_t)
            total_time += time_taken

        avg_hitting_times.append(total_time / samples)

    return avg_hitting_times

n_range = range(5, 15)  # Range of graph sizes
samples = 100           # Number of samples for each size
gamma = 1.0             # Transition rate
delta_t = 0.1           # Time step

# Start timing
start_time = time.time()

# Call the function
average_times = average_hitting_time_for_range2(n_range, samples, gamma, delta_t)

# End timing
end_time = time.time()

# Calculate and print the elapsed time
elapsed_time = end_time - start_time
print(f"Elapsed time: {elapsed_time} seconds")


In [ ]:
import cupy as cp
import networkx as nx

def probability_distribution(M_t, init_state):
    p0 = cp.zeros(len(M_t))
    p0[init_state] = 1
    p_t = cp.round(cp.dot(M_t, p0), 5)
    return p_t

def efficient_transition_matrix(eigenvalues, eigenvectors, t):
    D_t = cp.diag(cp.exp(eigenvalues * t))
    return cp.matmul(cp.matmul(eigenvectors, D_t), cp.linalg.inv(eigenvectors))

def gpu_random_choice(probs):
    cumulative = cp.cumsum(probs)
    rand = cp.random.rand()
    return cp.where(cumulative >= rand)[0][0]

def first_passage_time2(G, gamma, start, end, delta_t):
    D = cp.diag(cp.array(list(dict(G.degree()).values())))
    A = cp.array(nx.to_numpy_array(G))
    L = -gamma * (D - A)

    # Compute eigenvalues and eigenvectors using CuPy
    eigenvalues, eigenvectors = cp.linalg.eigh(L)

    current_state = start
    time_elapsed = delta_t
    
    while current_state != end:
        M_t = efficient_transition_matrix(eigenvalues, eigenvectors, time_elapsed)
        p_t = probability_distribution(M_t, current_state)
        
        neighbors = cp.array(list(G.neighbors(current_state)) + [current_state])
        transition_probabilities = p_t[neighbors]

        if transition_probabilities.sum() > 0:
            transition_probabilities /= transition_probabilities.sum()
        else:
            transition_probabilities = cp.ones(len(neighbors)) / len(neighbors)

        next_state = np.random.choice(neighbors, p=transition_probabilities)
        current_state = next_state.item()  # Convert CuPy scalar to Python scalar
        time_elapsed += delta_t

    return time_elapsed


In [ ]:
import time

def average_hitting_time_for_range2(n_range, samples, gamma, delta_t):
    """
    Calculates the average hitting time for graphs of sizes within n_range,
    performing a specified number of samples for each graph size.

    :param n_range: Range of graph sizes to test (e.g., range(5, 10))
    :param samples: Number of samples to perform for each graph size
    :param gamma: Transition rate parameter for the random walk
    :param delta_t: Time step for the random walk
    :return: List with average hitting times for each graph size
    """
    avg_hitting_times = []

    for n in n_range:
        # print(n)
        total_time = 0
        end_vertex = n // 2

        for _ in range(samples):
            G = nx.cycle_graph(n)
            start_vertex = 0

            time_taken = first_passage_time2(G, gamma, start_vertex, end_vertex, delta_t)
            total_time += time_taken

        avg_hitting_times.append(total_time / samples)

    return avg_hitting_times

n_range = range(5, 15)  # Range of graph sizes
samples = 100           # Number of samples for each size
gamma = 1.0             # Transition rate
delta_t = 0.1           # Time step

# Start timing
start_time = time.time()

# Call the function
average_times = average_hitting_time_for_range2(n_range, samples, gamma, delta_t)

# End timing
end_time = time.time()

# Calculate and print the elapsed time
elapsed_time = end_time - start_time
print(f"Elapsed time: {elapsed_time} seconds")
